In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Dense, concatenate

# Make NumPy printouts easier to read.
np.set_printoptions(precision=5, suppress=True)

import tensorflow as tf
print(tf.__version__)

2.15.0


In [27]:
dataset = pd.read_csv('data_obrobiona/calosc.csv')

dataset

[[ 4.       0.0001  19.75   ]
 [ 4.       0.0002   8.875  ]
 [ 4.       0.0003  19.8125 ]
 ...
 [ 2.       0.9999  15.625  ]
 [ 2.       0.99995  4.5    ]
 [ 2.       1.      15.6875 ]]


In [10]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
x_train=train_dataset.drop(columns="przeplyw");
x_test=test_dataset.drop(columns="przeplyw");
y_train=train_dataset.przeplyw;
y_test=test_dataset.przeplyw;
train_data=x_train.drop(columns="czas")
train_czas=x_train.drop(columns="dzien_tygodnia")

In [11]:
train_czas.shape

(295607, 1)

In [12]:
model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(126, activation='relu', input_shape=(2,)),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(1,activation='linear') 
])

In [13]:


model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),
              loss="mse",
              metrics=['mae'])

In [14]:
model.fit(x_train, y_train, epochs=5, batch_size=32)

Epoch 1/5


9238/9238 [==============================] - 24s 2ms/step - loss: 21.8194 - mae: 4.0263
Epoch 2/5
9238/9238 [==============================] - 23s 2ms/step - loss: 21.3640 - mae: 3.9984
Epoch 3/5
9238/9238 [==============================] - 20s 2ms/step - loss: 21.3492 - mae: 3.9976
Epoch 4/5
9238/9238 [==============================] - 21s 2ms/step - loss: 21.3288 - mae: 3.9961
Epoch 5/5
9238/9238 [==============================] - 21s 2ms/step - loss: 21.3194 - mae: 3.9959


In [15]:
model.evaluate(x_test,  y_test, verbose=2)

2310/2310 - 4s - loss: 21.5347 - mae: 4.0164 - 4s/epoch - 2ms/step


[21.534719467163086, 4.016350269317627]

In [16]:
sample_time=2;      #[s]
symulation_time=60; #[min]

current_hours=23;   #[h]
current_minutes=50; #[min]
current_day=1;      #[label, pon-1, wt-2,...,nd-7]

In [17]:
sample_time_normalized=sample_time/(60*60*24);              #[min - normalized]
symulation_time_normlized=symulation_time/(24*60)           #[min - normalized]
current_time=(current_hours*60+current_minutes)/(24*60)     #[min - normalized]

endtime=current_time+symulation_time_normlized;
if endtime<1:
    flow_time=np.arange(current_time,endtime,sample_time_normalized)                    #[min - normalized]
    flow_time=np.vstack((current_day*np.ones_like(flow_time, dtype=int),flow_time)).T   #[label, min]
else:
    temp_1=np.arange(current_time,1,sample_time_normalized)                             #[min - normalized]
    temp_1=np.vstack((current_day*np.ones_like(temp_1, dtype=int),temp_1)).T            #[label, min]

    temp_2=np.arange(0,endtime-1,sample_time_normalized)                                #[min - normalized]
    temp_2=np.vstack(((current_day+1)*np.ones_like(temp_2, dtype=int),temp_2)).T        #[label, min] 

    flow_time=np.concatenate((temp_1,temp_2))                                           #[label, min]
flow_time

array([[1.     , 0.99306],
       [1.     , 0.99308],
       [1.     , 0.9931 ],
       ...,
       [2.     , 0.03468],
       [2.     , 0.0347 ],
       [2.     , 0.03472]])

In [18]:
flow_result = model.predict(flow_time)                   # [l/min]
water_usage_predict=np.sum(flow_result)*(sample_time/60) # [l]

water_usage_predict

57/57 [==============================] - 0s 2ms/step


700.1501302083333

In [25]:
# print(flow_result[5])
y_test

1          8.8750
4          4.5000
21        14.8750
22        10.1875
27         6.3125
           ...   
369486    11.2500
369494    12.3125
369497    12.1875
369498    16.2500
369499    19.5625
Name: przeplyw, Length: 73902, dtype: float64